# Comparing SISO CDL Channel results with Matlab
Compare the results of this notebook with the Matlab file ``SisoCdl.mlx`` in the ``MatlabFiles`` directory.

The ".mat" files in the ``MatlabFiles`` directory were created by Matlab running the ``SisoCdl.mlx`` file. If you want to recreate these files, follow the instructions in the Matlab file. [Here](MatlabFiles/SisoCdl.html) is the execution results of this code in Matlab.

In [1]:
import numpy as np
import scipy.io

from neoradium import Carrier, CdlChannel, random, Waveform
from neoradium.utils import getNmse

matlabFilesPath = "./MatlabFiles"

In [2]:
carrier = Carrier(startRb=0, numRbs=25, spacing=15)    # Carrier 25 Resource Blocks, 15KHz subcarrier spacing
bwp = carrier.curBwp                                   # The only bandwidth part in the carrier

cdlModel = 'C'
seed = 123
phiInit, coupling = CdlChannel.getMatlabRandomInit(cdlModel, seed) # Match Matlab random values

speedKmh = 15                # speed of UE device: 15 km/h
speed = speedKmh*1000/3600   # m/s
c = 299792458                # Speed of light
fc = 4e9                     # 4 GHz
d = speed*fc/c               # Doppler Shift (Hz)

# NOTE: Always make sure to use this antenna config in Matlab to match the standard and NeoRadium's default
# cdl.TransmitAntennaArray.Size = [1 1 1 1 1];     % 1x1 antenna panel
# cdl.TransmitAntennaArray.Element = '38.901';     % Make sure we are using 3GPP antennas. (Default is not 3GPP)
# cdl.TransmitArrayOrientation = [0; 0; 0];        % Transmit antenna orientation bearing(α), downtilt(β), and slant(γ) angles in degrees
# cdl.TransmitAntennaArray.PolarizationAngles = [0 90];

# cdl.ReceiveAntennaArray.Size = [1 1 1 1 1];     % 1x1 antenna
# cdl.ReceiveAntennaArray.Element = '38.901';     % Make sure we are using 3GPP antennas. (Default is not 3GPP)
# cdl.ReceiveArrayOrientation = [180; 0; 0];      % Receive antenna orientation bearing(α), downtilt(β), and slant(γ) angles in degrees
# cdl.ReceiveAntennaArray.PolarizationAngles = [0 90];


# Create the channel model
channel = CdlChannel(bwp, cdlModel, delaySpread=10, carrierFreq=fc, dopplerShift=d,
                     initialPhases = phiInit, rayCoupling = coupling,
                     stopBandAtten = 70)            # NeoRadium default is 80; Matlab default is 70;
channel.print()


CDL-C Channel Properties:
  carrierFreq:          4 GHz
  normalizeGains:       True
  normalizeOutput:      True
  txDir:                Downlink
  filterLen:            16 samples
  delayQuantSize:       64
  stopBandAtten:        70 dB
  dopplerShift:         55.59401586635868 Hz
  coherenceTime:        7.611 milliseconds
  delaySpread:          10 ns
  ueDirAZ:              0.0°, 90.0°
  Cross Pol. Power:     7 dB
  angleSpreads:         2° 15° 3° 7°
  TX Antenna:
    freqRange:                   0 Hz .. 100 GHz
    polAngle:                    0°
    polModel:                    2
    beamWidth:                   65°,65°
    verticalSidelobeAttenuation: 30
    maxAttenuation:              30 dB
    mainMaxGain:                 8 dBi
  RX Antenna:
    freqRange:                   0 Hz .. 100 GHz
    polAngle:                    0°
    polModel:                    2
    beamWidth:                   65°,65°
    verticalSidelobeAttenuation: 30
    maxAttenuation:              30 dB
 

In [3]:
# Create a random signal for 1 subframe (1 ms)
t = 0.001       # 1 subframe = 1 ms
numInputSamples = int(channel.sampleRate * t)
nr, nt = channel.nrNt     # Get the number of antanna from the channel

# Load the "txWaveform" generated by the Matlab code
txWaveform = scipy.io.loadmat(matlabFilesPath+'/txWaveform.mat')['txWaveform'].T
assert txWaveform.shape==(nt, numInputSamples)

# Check the following numbers with the Matlabmatlab-generated numbers to make sure we are using
# the same input signal:
print("TX Waveform Data:\n", np.round(txWaveform[0,200:204],4))  # Matlab: txWaveform(201:204,3:4)

# Use the following line instead of above line to create a random signal (The result will be 
# different from Matlab)
# txWaveform = np.random.normal(size=(numSamples, nt)) + 1j*np.random.normal(size=(numSamples, nt))

# Now apply the channel to the waveform
rxWaveform = channel.applyToSignal(txWaveform)  
print("RX Waveform Data:\n", np.round(rxWaveform[0,200:204],4))  # Matlab: rxWaveform(201:204,2)

# Load Matlab results and compare with the above results
rxWaveformMatlab = scipy.io.loadmat(matlabFilesPath+'/rxWaveform.mat')['rxWaveform']
assert rxWaveformMatlab.shape==(numInputSamples, nr)
print("NMSE:", getNmse(rxWaveformMatlab.T,rxWaveform.waveform))   # NMSE between NeoRadium and Matlab results

TX Waveform Data:
 [ 0.3957+0.3554j  0.1609-1.6695j -0.1543+0.4127j -0.484 -1.0458j]
RX Waveform Data:
 [ 1.0979-0.9033j  0.1692-0.5916j -0.7662-0.9718j  0.1534-0.7265j]
NMSE: 9.313019773677275e-05
